# Emissions to forcing for aerosol

We will continue building up species by species, taking the notebook before as a template.

In [ ]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import warnings

from fair21.forcing.ghg import ghg
from fair21.constants.general import TIME_AXIS, SPECIES_AXIS
from fair21.constants.gases import molwt, burden_per_emission, lifetime
from fair21.defaults import slcf_list, gas_list, montreal_gas_list, minor_gas_list, n_gas_boxes
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon,
    radiative_efficiency
)
from fair21.defaults.forcing import tropospheric_adjustment
from fair21.defaults.short_lived_forcers import pre_industrial_emissions

from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha
)
from fair21.gas_cycle.forward import step_concentration

from fair21.constants.gases import BR_ATOMS, CL_ATOMS
from fair21.defaults.ozone import radiative_efficiency as o3_radiative_efficiency, br_cl_ratio, fractional_release
from fair21.defaults.aerosol import radiative_efficiency as ari_radiative_efficiency, beta, shape
from fair21.defaults.forcing import tropospheric_adjustment
from fair21.defaults.gases import pre_industrial_concentration
from fair21.defaults.short_lived_forcers import pre_industrial_emissions
from fair21.defaults import gas_list, slcf_list

from fair21.forcing.ozone import thornhill_skeie

In [ ]:
ari_radiative_efficiency

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']
species_list = gas_list + slcf_list
species_list.extend(['O3', 'Aerosol-cloud interactions'])
species_list

In [ ]:
# initialise dicts for outputs. These will eventually be moved to the forward model
concentration = {}
airborne_emissions = {}
cumulative_emissions = {}
effective_radiative_forcing = {}

n_timesteps = 751
n_scenarios = len(scenarios)

# kicker for ozone
n_species = len(species_list)
n_species

In [ ]:
# grab some emissions
species_index_mapping = {}
emissions_array = np.ones((n_scenarios, n_species, n_timesteps, 1)) * np.nan
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for iscen, scenario in enumerate(scenarios):
    for ispec, specie in enumerate(species_list):
        species_index_mapping[specie] = ispec
        specie_rcmip_name = specie.replace("-", "")
        try:
            emissions_array[iscen, ispec, ...] = df.loc[
                (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+specie_rcmip_name)) & (df['Region']=='World'), '1750':
            ].interpolate(axis=1).values.T
        except ValueError:
            warnings.warn("{} was not found in emissions file".format(specie))
    
        # CO2 and N2O units need to behave
        if specie in ('CO2', 'N2O'):
            emissions_array[iscen, ispec, :, :] = emissions_array[iscen, ispec, :, :] / 1000

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp585-mean-temperature.csv')
ssp585_temperature_rfmip = df['temperature'].values

In [ ]:
# initialise arrays. Using arrays makes things run quicker

# remember: scen, species, time, box
# where we don't care about keeping outputs we use a singleton dimension to maintain bookkeeping
# possible memory saver: differentiate between ensembles where we vary emissions and those where we vary climate parameters

# this scenario definition needs to be part of the initialisation, either from pyam, scmdata or fair emissions files
concentration_array = np.ones((n_scenarios, n_species, n_timesteps, 1)) * np.nan
g0 = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
g1 = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
alpha_lifetime = np.ones((n_scenarios, n_species, 1, 1))
airborne_emissions = np.zeros((n_scenarios, n_species, 1, 1))
gas_boxes = np.zeros((n_scenarios, n_species, 1, n_gas_boxes))
iirf_0_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
iirf_cumulative_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
iirf_temperature_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
iirf_airborne_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
burden_per_emission_array = np.ones((1, n_species, 1, 1)) * np.nan
lifetime_array = np.ones((n_scenarios, n_species, 1, n_gas_boxes)) * np.nan
pre_industrial_emissions_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
pre_industrial_concentration_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
partition_fraction_array = np.zeros((n_scenarios, n_species, 1, n_gas_boxes))
natural_emissions_adjustment_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan

cumulative_emissions_array = np.cumsum(emissions_array, axis=TIME_AXIS)

# this one for the general forcing
tropospheric_adjustment_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
radiative_efficiency_array = np.ones((n_scenarios, n_species, 1, 1)) * np.nan
erf = np.ones((n_scenarios, n_species, n_timesteps, 1)) * np.nan

# ozone
fractional_release_array = np.zeros((1, n_species, 1, 1)) # gonna assume this doesn't change
cl_atoms_array = np.zeros((1, n_species, 1, 1))
br_atoms_array = np.zeros((1, n_species, 1, 1))
o3_radiative_efficiency_array = np.zeros((1, n_species, 1, 1))

# aerosols
ari_radiative_efficiency_array = np.zeros((n_scenarios, n_species, 1, 1))

for ispec, specie in enumerate(species_list):
    # options pertaining to SLCFs
    if specie in slcf_list:
        pre_industrial_emissions_array[:, ispec, :, :] = pre_industrial_emissions[specie]
        ari_radiative_efficiency_array[:, ispec, :, :] = ari_radiative_efficiency[specie]["AR6"]
    
    # options pertaining to GHGs
    if specie in gas_list:
        lifetime_array[:, ispec, :, :] = lifetime[specie]
        partition_fraction_array[:, ispec, :, :] = partition_fraction[specie]
        iirf_0_array[:, ispec, :, :] = iirf_0[specie]
        iirf_cumulative_array[:, ispec, :, :] = iirf_cumulative[specie]
        iirf_temperature_array[:, ispec, :, :] = iirf_temperature[specie]
        iirf_airborne_array[:, ispec, :, :] = iirf_airborne[specie]
        burden_per_emission_array[:, ispec, :, :] = burden_per_emission[specie]
        pre_industrial_concentration_array[:, ispec, :, :] = pre_industrial_concentration[specie]
        partition_fraction_array[:, ispec, :, :] = partition_fraction[specie]
        natural_emissions_adjustment_array[:, ispec, :, :] = natural_emissions_adjustment[specie]
        fractional_release_array[:, ispec, :, :] = fractional_release[specie]
        br_atoms_array[:, ispec, :, :] = BR_ATOMS[specie]
        cl_atoms_array[:, ispec, :, :] = CL_ATOMS[specie]
        
    # options pertaining only to minor GHGs
    if specie in minor_gas_list:
        radiative_efficiency_array[:, ispec, :, :] = radiative_efficiency[specie]

    # options pertaining to all species
    tropospheric_adjustment_array[:, ispec, :, :] = tropospheric_adjustment[specie]
    o3_radiative_efficiency_array[:, ispec, :, :] = o3_radiative_efficiency[specie]

g0, g1 = calculate_g(lifetime_array, partition_fraction_array)